# Setup


In [ ]:
import weaviate, os

# Connect to a cloud instance of Weaviate (with WCS)
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
# )

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local()

client.is_ready()

## Create Tenant-ready collection

In [ ]:
from weaviate.classes.config import Configure

if (client.collections.exists("Play")):
    client.collections.delete("Play")

client.collections.create(
    "Play",
    vectorizer_config=Configure.Vectorizer.none(),

    multi_tenancy_config=Configure.multi_tenancy(True)
)

## Create tenants
> tenant name – must be made of alphanumeric characters (a-z, A-Z, 0-9), underscore (_), and hyphen (-), with a length between 1 and 64 characters'


In [ ]:
from weaviate.classes.tenants import Tenant

play = client.collections.get("Play")

play.tenants.create([
    Tenant(name="ten_A"),
    Tenant(name="ten_B"),
    Tenant(name="ten_C"),
    Tenant(name="ten_D"),
])

## List Tenants

In [ ]:
play.tenants.get()

## Access Tenants

In [ ]:
# this will fail – multi-tenant collections require us to use tenants
play.aggregate.over_all()

In [ ]:
play = client.collections.get("Play")
tenA = play.with_tenant("ten_A")

tenA.aggregate.over_all()

### Insert data

In [ ]:
# tenA = client.collections.get("Play").with_tenant("ten_A")
play = client.collections.get("Play")
tenA = play.with_tenant("ten_A")

tenA.data.insert_many([
    {
       "title": "A book about vector databases"
    },
    {
       "title": "Tutorial for multimodal collections"
    },
])

tenA.aggregate.over_all()

### Query Example

In [ ]:
from weaviate.classes.query import Filter

tenA = client.collections.get("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

### Delete Tenants

In [ ]:
play.tenants.remove(["ten_D"])
play.tenants.get()

### Update Tenants – HOT & COLD
Tenants can be:
* `HOT` (default) - active tenant
* `COLD` - inacative, cannot be search on, their index is not loaded into memory

In [ ]:
from weaviate.classes.tenants import Tenant, TenantActivityStatus

play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.COLD),
    Tenant(name="ten_C", activity_status=TenantActivityStatus.HOT),
])

play.tenants.get()

### Cannot search `COLD` tenants

In [ ]:
# tenA = client.collections.get("Play").with_tenant("ten_A")

response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

### But you can turn them `HOT` again

In [ ]:
play.tenants.update([
    Tenant(name="ten_A", activity_status=TenantActivityStatus.HOT),
])

play.tenants.get()

In [ ]:
response = tenA.query.fetch_objects(
    filters=Filter.by_property("title").like("about")
)

for item in response.objects:
    print(item.properties)

## Clean up

In [ ]:
client.collections.delete("Play")